In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("diabetes.csv")

In [17]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
df.shape

(768, 9)

In [19]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [20]:
X = df.iloc[:,:8]
y = df.iloc[:,-1]

In [21]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [22]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=30)

In [24]:
from sklearn.impute import SimpleImputer 
impute = SimpleImputer(missing_values=0,strategy="mean")
X_train = impute.fit_transform(X_train)
X_test = impute.fit_transform(X_test)

In [27]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
rf = RandomForestClassifier()

In [28]:
#hyperparameter tuning
n_estimators = [int(i) for i in np.linspace(100,1000,10)]
criterion = ['gini','entropy']
max_depth = [int(i) for i in np.linspace(100,1000,10)]
min_samples_split = [2,3,5,6,8,9]
min_samples_leaf = [1,2,8,6,5,7]
max_features = ['auto','sqrt','log2']

param_grid = {'n_estimators':n_estimators,
              'criterion':criterion,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'max_features':max_features
}
print(param_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'criterion': ['gini', 'entropy'], 'max_depth': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'min_samples_split': [2, 3, 5, 6, 8, 9], 'min_samples_leaf': [1, 2, 8, 6, 5, 7], 'max_features': ['auto', 'sqrt', 'log2']}


In [29]:
from sklearn.model_selection import RandomizedSearchCV
rf_cv = RandomizedSearchCV(estimator=rf,param_distributions=param_grid,n_iter=100,verbose=2,cv=3,n_jobs=-1)
rf_cv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [30]:
best_param = rf_cv.best_estimator_

In [31]:
y_pred = best_param.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:{}".format(accuracy_score(y_test,y_pred)))

[[137  22]
 [ 24  48]]
Accuracy Score:0.8008658008658008


In [33]:
import pickle
file = open("random_forest_diabetes_model.pkl","wb")
pickle.dump(rf_cv,file)